In [ ]:
# http://stackoverflow.com/questions/15048963/alternative-to-the-deprecated-setup-environ-for-one-off-django-scripts
import os
import django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "budjetti.settings")
#from django.conf import settings

In [ ]:
import django
django.setup()

In [ ]:
import glob
from balanssi.models import Transaction

In [ ]:
from balanssi import data_util

In [ ]:
data_util.delete_all_transactions()

In [ ]:
# Import data
fnames = sorted(glob.glob('/tmp/Tapahtumat*.txt'))
for fname in fnames:
    data_util.load_data(fname)

In [ ]:
sorted(map(str, Transaction.objects.filter(category=None)))

In [ ]:
#Transaction.objects.filter(category=None, party='ANTEROINEN JOHANNA')[0].action

In [ ]:
len(Transaction.objects.all())

In [ ]:
len(Transaction.objects.filter(category=None))

In [ ]:
import pandas as pd

In [ ]:
d = pd.read_csv('http://localhost:8000/balanssi/transactions.csv')

In [ ]:
import dateutil
d['date'] = d['pay_date'].apply(dateutil.parser.parse)

In [ ]:
d

In [ ]:
import matplotlib.pyplot as plt
plt.close('all')

In [ ]:
a = d.groupby('category')['amount'].sum()

In [ ]:
d_ = d

In [ ]:
d.category
d[(d.amount < 0) |~d.category.str.contains('outgoing').fillna(False)].category.unique()

In [ ]:
d[d.category == 'root > expenses']

In [ ]:
%matplotlib notebook

In [ ]:
d['year'] = d.date.map(lambda x: x.year)
d['month'] = d.date.map(lambda x: x.month)
#d['month_number'] = d.date.map(lambda x: (x.year - 2013) * 12 + x.month)
mask = (d.amount < 0) & ~d['category'].str.contains('outgoing transfers').fillna(False)
g = d[mask].set_index('date').groupby(['year', 'month'])


for (year, month), gdata in g:
    plt.figure()#(figsize=(24,16))
    names = gdata.groupby('category')['amount'].sum().index
    vals = gdata.groupby('category')['amount'].sum().values
    tot = vals.sum()
    lbl = '{}/{} {:.0f} EUR'.format(year, month, tot)
    plt.pie(-vals, labels=names, autopct='%1.1f%%')
    plt.gca().set_aspect('equal')
    plt.title(lbl)
    plt.tight_layout()

In [ ]:
plt.close('all')

In [ ]:
plt.figure()
xx = d[mask].groupby('category')['amount'].sum()
names = xx.index
vals = xx.values
plt.pie(-vals, labels=names, autopct='%1.1f%%')
plt.gca().set_aspect('equal')
plt.title('all time')
plt.tight_layout()
plt.show()

In [ ]:
# plotting categories as time series
a = d.set_index('date').groupby(['year', 'month', 'category'])['amount'].apply(sum)
b = a.reset_index(['year', 'month'],drop= True)
for cat, vals in b.groupby(b.index):
    #if 'restaurant' not in cat: continue
    if 'groceries' not in cat: continue
    plt.figure()#(figsize=(24,16))
    plt.plot(vals, label=cat)
    plt.title(cat)

In [ ]:
a

In [ ]:
for a, b in d.set_index('month', drop=False).groupby(d.index).sum().groupby(['month', 'category']):
    print b.index

In [ ]:
for k, ((cat_name), group) in enumerate(d.set_index('month').groupby(['category'])):
    print 'name',cat_name
    plt.figure()
    print group['amount']
    if k > 5:
        break
    #plt.plot(group.sum()['amount'])

In [ ]:
d2 = pd.read_csv('http://localhost:8000/balanssi/transactions.csv?amount__gte=0&groupby=category')

In [ ]:
d2